In [ ]:
library("phyloseq"); packageVersion("phyloseq")
library("ggplot2")
library("ape")
library("vegan"); packageVersion("vegan")
library("dplyr")
library("plyr"); packageVersion("plyr")
library("microbiome") 
suppressMessages(library(cowplot)); packageVersion("cowplot")
library(grid)
library(gridExtra)
library(ggpubr)


In [ ]:
#load files, OTU table, metadata, taxonomy, and tree file
#removed "#"and removes space from OTU ID, and removed first row on excel
SV_C <- t(read.table("microbiome/export_AC_DRMB/table-fil-BacArc_DRMB1AC.txt", 
                   row.names = 1, header = TRUE))
head(SV_C)
dim(SV_C)

SV_P <- t(read.table("microbiome/export_AP_DRMB//table-fil-BacArc_DRMB1AP.txt", 
                   row.names = 1, header = TRUE))
head(SV_P)
dim(SV_P)

#removed "#" from first row
map<- read.table("DR_MB1_meta_seqID_run1.txt", 
                 row.names = 1, header = TRUE)
#dim(map)
#head(map)


#ead(map)
## to import taxonomy I had to convert ; into tabs and remove last columns
tax <-as.matrix(read.table("exported_DRMB1/tax-no-BacArc_DRMB1_fill.txt",  
                           row.names = 1, header = FALSE, sep = "\t"))

colnames(tax) <- c ("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")
tax[tax==""] <- NA
head(tax)
dim(tax)

#head(tax)
tree_file <-read_tree("exported_DRMB1/tree_BacArc_DRMB1.nwk")
#head(tree_file)


In [ ]:
#import palmata data into phyloseq object
ps_pal = phyloseq(otu_table(SV_P, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_pal

#import cervicornis data into phyloseq object
ps_cer = phyloseq(otu_table(SV_C, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_cer

#Combine both species into one phyloseq object
ps = merge_phyloseq(ps_cer, ps_pal)
ps

In [ ]:
#tranform counts to CLR
#for both species
ps_clr <- microbiome::transform(ps, 'clr')
ps_clr 
#for cervicornis
ps_cer_clr <- microbiome::transform(ps_cer, 'clr')
ps_cer_clr
#for palmata
ps_pal_clr <- microbiome::transform(ps_pal, 'clr')
ps_pal_clr

In [ ]:
#tranform counts to relative abundance
#for cervicornis
psr_cer= transform_sample_counts(ps_cer, function(x) x / sum(x) )
psr_cer
#for palmata                               
psr_pal= transform_sample_counts(ps_pal, function(x) x / sum(x) )
psr_pal

In [ ]:
#ordinate all
psr_clr.ord <- ordinate(ps_clr, "RDA", "euclidean")

#plot ordination
PCA_pal_cer = plot_ordination(ps_clr, psr_clr.ord, label= "Genotype",
                              type="samples", color="Host_Species",
                              shape="Treatment", axes = c(1, 
    2)) +
scale_colour_manual(values = c("#56B4E9", "#E69F00")) +
theme_classic() +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 


PCA_pal_cer

In [ ]:
#load files, OTU table, metadata, taxonomy, and tree file
#removed "#"and removes space from OTU ID, and removed first row on excel
SV <- t(read.table("/microbiome/exported_DRMB1/feature-tableBacArc_DRMB1_fill.txt", row.names = 1, header = TRUE))
#head(SV)
#removed "#" from first row
map<- read.table("DR_MB1_meta_seqID_run1.txt", 
                 row.names = 1, header = TRUE)
#head(map)
## to import taxonomy I had to convert ; into tabs and remove last columns
tax <-as.matrix(read.table("microbiome/exported_DRMB1/tax-no-BacArc_DRMB1.txt",  
                           row.names = 1, header = FALSE, sep = "\t"))
#head(tax)
tree_file <-read_tree("microbiome/exported_DRMB1/tree_BacArc_DRMB1.nwk")
#head(tree_file)

colnames(tax) <- c ("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species")
tax[tax==""] <- NA
#head(tax)
dim(tax)

In [ ]:
#import non-filtered ASV table into phyloseq
ps_all = phyloseq(otu_table(SV, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_all


In [ ]:
#tranform into relative abundance
psr_all= transform(ps_all, "compositional")

In [ ]:
#make into df to use on ggplot
ps_sum_genus_all <- psr_all %>%
  tax_glom(taxrank = "Genus") %>%                     # agglomerate at genus level
  psmelt()    

In [ ]:
map$Host_Species

In [ ]:
#Make bar graphs with negative control
bar_neg= ggplot(subset(ps_sum_genus_all, Abundance > 0.05), 
       aes(x = Host_Species, y=Abundance)) + 
geom_bar(stat="identity", aes(fill = Genus)) +
scale_fill_manual(values=c("#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD",
                           "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", 
                           "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4",
                           "lightblue4", "lightpink", "ivory4","royalblue4", "darkorchid", 
                           "palevioletred1", "#56B4E9","#CBD588","yellow2","#5F7FC7", "orange","#DA5724",
                           "#CD9BCD", "gray80",
                           "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F",
                          "#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD")) +
guides(fill = guide_legend(keywidth = 0.3, , keyheight =.70, ncol=1)) +
theme_classic() +
theme(legend.text =element_text(size=5)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 8)) +
theme(axis.title.y = element_text(size = 8)) +
theme(axis.text.x = element_text(size =6)) +
theme(axis.text.y = element_text(size =6)) +
theme(strip.text.x = element_text(size =5)) +
 ylab("Relative abundance")
bar_neg

bar_neg_fill= ggplot(subset(ps_sum_genus_all, Abundance > 0.05), 
       aes(x = Host_Species, y=Abundance)) + 
geom_bar(stat="identity", position="fill", aes(fill = Genus)) +
scale_fill_manual(values=c("#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD",
                           "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", 
                           "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4",
                           "lightblue4", "lightpink", "ivory4","royalblue4", "darkorchid", 
                           "palevioletred1", "#56B4E9","#CBD588","yellow2","#5F7FC7", "orange","#DA5724",
                           "#CD9BCD", "gray80",
                           "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F",
                          "#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD")) +
guides(fill = guide_legend(keywidth = 0.3, , keyheight =.70, ncol=1)) +
theme_classic() +
theme(legend.text =element_text(size=5)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 8)) +
theme(axis.title.y = element_text(size = 8)) +
theme(axis.text.x = element_text(size =6)) +
theme(axis.text.y = element_text(size =6)) +
theme(strip.text.x = element_text(size =5)) +
 
ylab("Relative abundance")

bar_neg_fill

In [ ]:
sup1 = ggarrange(bar_neg_fill, bar_neg,
                  common.legend = TRUE, legend = "right",
                labels = c("A", "B"), font.label = list(size = 9) 
               )
sup1

In [ ]:
#save plot
ggsave("microbiome/Figures/Sup1.png",
       sup1, width = 7, height = 5, units = "in", dpi=300)

In [ ]:
#make bar graph without negtive 
bar_species = ggplot(subset(ps_sum_genus_all, Abundance > 0.05 & Genotype!="NaN"), 
       aes(x = Genotype, y=Abundance)) + 
geom_bar(stat="identity", position="fill", aes(fill = Genus)) +
scale_fill_manual(values=c("#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD",
                           "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", 
                           "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4",
                           "lightblue4", "lightpink", "ivory4","royalblue4", "darkorchid", 
                           "palevioletred1", "#56B4E9","#CBD588","yellow2","#5F7FC7", "orange","#DA5724",
                           "#CD9BCD", "gray80",
                           "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F",
                          "#56B4E9","#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD")) +
facet_grid(Treatment~Host_Species, scales="free") +
theme_classic() +
ylab("Relative Abundance > 0.05") + 
guides(fill = guide_legend(keywidth = 0.3, , keyheight =.40, ncol=1)) +
theme(legend.text =element_text(size=4)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 8)) +
theme(axis.title.y = element_text(size = 8)) +
theme(axis.text.x = element_text(size =4)) +
theme(axis.text.y = element_text(size =5)) +
theme(strip.text.x = element_text(face = "italic", size =7)) +
theme(strip.text.y = element_text(size =7)) +
theme(axis.text.x = element_text(angle = 90, hjust = 1))
bar_species

In [ ]:
#boxplot of Shannon diversity 
alpha_geno= ggplot(subset(map, Host_Species == "palmata" | Host_Species== "cervicornis"),
       aes(x=Genotype, y= Shannon, fill=Host_Species)) +
scale_fill_manual(values=c("#56B4E9", "#E69F00")) +
geom_boxplot() +
geom_point(aes(shape= Treatment),position = position_jitterdodge(), alpha=0.5) +
theme_bw() +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.text =element_text(size=4)) +
theme(legend.title = element_text(size=5)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 
#theme(legend.position="top", legend.box = "vertical")

alpha_geno



In [ ]:
#boxplot of Evenness 
alpha_even= ggplot(subset(map, Host_Species == "palmata" | Host_Species== "cervicornis"),
       aes(x=Genotype, y= Evenness, fill=Host_Species)) +
scale_fill_manual(values=c("#56B4E9", "#E69F00")) +
geom_boxplot() +
geom_point(aes(shape= Treatment),position = position_jitterdodge(), alpha=0.5) +
theme_bw() +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.text =element_text(size=4)) +
theme(legend.title = element_text(size=5)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 
#theme(legend.position="none")

alpha_even

In [ ]:
fig1 = plot_grid(alpha_geno, PCA_pal_cer,alpha_even, bar_species,
              labels = c("A", "C", "B", "D"), ncol = 2, align="none",
                label_size=12)
 fig1                              

In [ ]:
#save figure
ggsave("microbiome/Figures/Fig1.png"
       ,fig1, width = 7, height = 5, units = "in", dpi=300)

In [ ]:
#ordinate
psr_cer_clr.ord <- ordinate(ps_cer_clr, "RDA",  "euclidian")

#plot ordinations
cbPalette <- c( "#009E73","#D55E00","#696969")
cer_pca_eli= plot_ordination(ps_cer_clr, psr_cer_clr.ord, 
                                type="samples",
                                color="Outcome",
                                label="Genotype",
                              
                                axes = c(2, 3)) +
geom_point(size=1) +
stat_ellipse(geom = "polygon", type="norm", alpha=0.2, aes(color=Outcome, fill=Outcome)) + 
theme_classic() +
scale_fill_manual(values = cbPalette) +
scale_color_manual(values = cbPalette) +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

cer_pca_eli

In [ ]:
# compute dissimilarity indices
dis_cer <- vegdist(otu_table(ps_cer_clr), method ="euclidean")
# PERMDISP2 procedure for the analysis of multivariate homogeneity of group dispersions (variances).
mod_cer <- betadisper(dis_cer, sample_data(ps_cer_clr)$Outcome)
mod_cer
mod_dist_cer= as.data.frame(mod_cer$distances)
dim(mod_dist_cer)
#head(mod_dist)
#dim(sample_data(ps_cer_clr))
#head(sample_data(ps_cer_clr))
mod_dist_df_cer=cbind(sample_data(ps_cer_clr), mod_dist_cer) 
head(mod_dist_df_cer)

In [ ]:
#plot variance 
cbPalette <- c( "#009E73","#696969", "#D55E00")
mod_dist_df_cer$Outcome <- factor(mod_dist_df_cer$Outcome, levels = c("Control",
                                                         "VisuallyUnaffected","Diseased"))



cer_box= ggplot(mod_dist_df_cer,
       aes(x=Outcome, y=mod_cer$distances)) +
geom_boxplot(aes(color=Outcome)) +
scale_color_manual(values = cbPalette) +
theme_linedraw() +
facet_grid(Host_Species~.) +
theme_classic() +
ylab("Distance to Centroid") +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

cer_box

In [ ]:
psr_pal_clr.ord <- ordinate(ps_pal_clr, "RDA",  "euclidian")

In [ ]:
cbPalette <- c( "#009E73","#D55E00","#696969")
pal_pca_eli= plot_ordination(ps_pal_clr, psr_pal_clr.ord, 
                                type="samples",
                                color="Outcome",
                             axes = c(1, 2),
                            label="Genotype") +
geom_point(size=1) +
stat_ellipse(geom = "polygon", type="norm", alpha=0.2, aes(color=Outcome, fill=Outcome)) + 
theme_classic() +
scale_fill_manual(values = cbPalette) +
scale_color_manual(values = cbPalette) +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

pal_pca_eli


In [ ]:
dis <- vegdist(otu_table(ps_pal_clr), method ="euclidean")
# PERMDISP2 procedure for the analysis of multivariate homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_pal_clr)$Outcome)

mod_dist= as.data.frame(mod$distances)
#dim(mod_dist)
#head(mod_dist)
#dim(sample_data(ps_cer_clr))
#head(sample_data(ps_cer_clr))
mod_dist_df=cbind(sample_data(ps_pal_clr), mod_dist) 


In [ ]:
cbPalette <- c( "#009E73","#696969", "#D55E00")
mod_dist_df$Outcome <- factor(mod_dist_df$Outcome, levels = c("Control",
                                                         "VisuallyUnaffected","Diseased"))



pal_box = ggplot(mod_dist_df,
       aes(x=Outcome, y=mod$distances)) +
geom_boxplot(aes(color=Outcome)) +
scale_color_manual(values = cbPalette) +
theme_linedraw() +
facet_grid(Host_Species~.) +
theme_classic() +
ylab("Distance to Centroid") +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

pal_box

In [ ]:
fig2 = ggarrange(cer_pca_eli, cer_box, pal_pca_eli, pal_box,
                 ncol = 2, nrow=2,
                  common.legend = TRUE, legend = "top",
                labels = c("A", "B", "C", "D"), font.label = list(size = 9)
               )
fig2                             

In [ ]:
ggsave("microbiome/Figures/Fig2.png"
       ,fig2, width = 5, height = 5, units = "in", dpi = 300)

In [ ]:
library(RColorBrewer)
colorpal <-  c("#CBD588","darkgreen", "orange","#DA5724","#CD9BCD", 
"#AD6F3B","#D14285", "#652926","#8569D5", "#5E738F","#D1A33D", "#8A7C64","lightsalmon",
    "aquamarine4", 
    "darkorchid", "palevioletred1","orange","#DA5724","#CD9BCD", "gray80")




cer_geno_ord= plot_ordination(ps_cer_clr, psr_cer_clr.ord, type="sample", color="Genotype",
                shape="Treatment", label = "Nursery"
               ) + theme_bw() +



theme_classic() +
scale_fill_manual(values = colorpal) +
scale_color_manual(values = colorpal) +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

cer_geno_ord

pal_geno_ord=plot_ordination(ps_pal_clr, psr_pal_clr.ord, 
                             type="sample", color="Genotype",
                shape="Treatment", label = "Nursery"
               ) + theme_bw() +



theme_classic() +
scale_fill_manual(values = colorpal) +
scale_color_manual(values = colorpal) +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) 

pal_geno_ord

In [ ]:
Sup2 =ggarrange(cer_geno_ord, pal_geno_ord,
                 ncol = 2, nrow=1,
                  
                labels = c("A", "B"), font.label = list(size = 9)
               )
Sup2

In [ ]:
ggsave("Figures/supplementalFig2.png"
       ,Sup2, width = 6, height = 4, units = "in", dpi = 300)

In [ ]:
#select ASVs that were significant for outcome from ANCOM results

GoodTaxa = c(
"e2829560ad1e796bebb56d909c7fb12d",
"5c518d771ed47c631781c4e204c796b0",
"fa87a3ed888b44bab9d77e04cbb99665")
allTaxa = taxa_names(psr_pal)
allTaxa <- allTaxa[(allTaxa %in% GoodTaxa)]
sig_pal = prune_taxa(allTaxa, psr_pal)
sig_pal

sum_genus_sig_pal  <- sig_pal  %>%
  tax_glom(taxrank = "Family") %>%                     # agglomerate at family level
  psmelt() 


GoodTaxa = c(
"7eb68c2ff12bb8a0a46d036c37f8f26e",
"fa87a3ed888b44bab9d77e04cbb99665")
allTaxa = taxa_names(psr_cer)
allTaxa <- allTaxa[(allTaxa %in% GoodTaxa)]
sig_cer = prune_taxa(allTaxa, psr_cer)
sig_cer

sum_genus_sig_cer  <- sig_cer  %>%
  tax_glom(taxrank = "Genus") %>%                     # agglomerate at genus level
  psmelt() %>%                                         # Melt to long format
  
head(sum_genus_sig_cer)

In [ ]:
sum_genus_sig_cer$Outcome <- factor(sum_genus_sig_cer$Outcome, levels = c("Control",
                                                         "VisuallyUnaffected","Diseased"))

box_sig_cer = ggplot(subset(sum_genus_sig_cer, Abundance > 0.0),
       aes(x=Outcome, y= Abundance, color =Family)) +
geom_boxplot() +
geom_point(position = position_jitterdodge(), alpha=0.5, size=1) +
scale_color_manual(values=c("#5F7FC7","#696969")) +
scale_y_continuous(trans='log10') +
theme_bw() +
#theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) +
theme(legend.position="none") +
ylab("Relative abundance")

box_sig_cer

In [ ]:
sum_genus_sig_pal$Outcome <- factor(sum_genus_sig_pal$Outcome, levels = c("Control",
                                                         "VisuallyUnaffected","Diseased"))


box_sig_pal =ggplot(subset(sum_genus_sig_pal, Abundance > 0),
       aes(x=Outcome, y= Abundance, color =Family)) +
geom_boxplot() +
geom_point(position = position_jitterdodge(), alpha=0.5, size=1) +
scale_color_manual(values=c("#8569D5","#CBD588","#5F7FC7", "orange","#DA5724",
                  "#CD9BCD", "gray80", "#AD6F3B", "#673770","#D14285",
                  "#652926")) +
scale_y_continuous(trans='log10') +
theme_bw() +
#theme(axis.text.x = element_text(angle = 90, hjust = 1))
theme(legend.text =element_text(size=5)) +
theme(legend.title = element_text(size=6)) +
theme(axis.title.x = element_text(size = 7)) +
theme(axis.title.y = element_text(size = 7)) +
theme(axis.text.x = element_text(size =5)) +
theme(axis.text.y = element_text(size =5)) +
theme(legend.position="none") +
ylab("Relative abundance")
box_sig_pal

In [ ]:
ps_sig =merge_phyloseq(sig_cer, sig_pal)
ps_sig


sum_ps_sig  <- ps_sig  %>%
  tax_glom(taxrank = "Family") %>%                     # agglomerate at genus level
  psmelt() %>%                                         # Melt to long format
  arrange(Family) 


In [ ]:
sum_ps_sig $Outcome <- factor(sum_ps_sig $Outcome, levels = c("Control",
                                                         "VisuallyUnaffected","Diseased"))
bar_sig_bac =ggplot(subset(sum_ps_sig, Abundance < 0.04), aes(x =Genotype_replicate, y=Abundance)) + 
geom_bar(stat="identity", aes(fill = Family)) +
scale_fill_manual(values=c("#8569D5","#CBD588", "#5F7FC7","#696969", "orange","#DA5724","#CD9BCD", "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4", "khaki1", "lightblue4", "lightpink", "ivory4", "yellow2","royalblue4", "darkorchid", "palevioletred1", "#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD", "gray80")) +
facet_grid(.~Outcome, scales="free") +
theme_classic() +
guides(fill = guide_legend(keywidth = 0.3, , keyheight =.40)) +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.position="top") +
theme(legend.text =element_text(size=4)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 6)) +
theme(axis.title.y = element_text(size = 5.5)) +
theme(axis.text.x = element_text(size =3.5)) +
theme(axis.text.y = element_text(size =5)) +
theme(strip.text.x = element_text(size =5)) +
xlab("Genotype") +
ylab("Relative abundance")

bar_sig_bac

In [ ]:
Fig3 =ggdraw() +
  draw_plot(box_sig_cer, x = 0, y = .5, width = .5, height = .5) +
  draw_plot(box_sig_pal, x = .5, y = .5, width = .5, height = .5) +
  draw_plot(bar_sig_bac, x = 0, y = 0, width = 1, height = 0.5) +
  draw_plot_label(label = c("A", "B", "C"), size = 10,
                  x = c(0, 0.5, 0), y = c(1, 1, 0.5)) 
Fig3

In [ ]:
ggsave("microbiome/Figures/Fig3.png"
       ,Fig3, width = 5, height = 4, units = "in", dpi=300)

In [ ]:
ggplot(subset(sum_ps_sig, Abundance < 0.04), aes(x =Genotype_replicate, y=Abundance)) + 
geom_bar(stat="identity", aes(fill = Family)) +
scale_fill_manual(values=c("#8569D5","#CBD588", "#5F7FC7","#696969", "orange","#DA5724","#CD9BCD", "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4", "khaki1", "lightblue4", "lightpink", "ivory4", "yellow2","royalblue4", "darkorchid", "palevioletred1", "#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD", "gray80")) +
facet_grid(Outcome~Susceptibility, scale="free_x") +
theme_classic() +
guides(fill = guide_legend(keywidth = 0.3, , keyheight =.40)) +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.position="top") +
theme(legend.text =element_text(size=4)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 6)) +
theme(axis.title.y = element_text(size = 5.5)) +
theme(axis.text.x = element_text(size =3.5)) +
theme(axis.text.y = element_text(size =5)) +
theme(strip.text.x = element_text(size =5)) 

In [ ]:
#select ASVs that were significant for genotype from ANCOM results
GoodTaxa = c(
"3f8018e35043db82841f695939b95ece",
"87624cf95e61dd0ce58bab24686f1eb4",
"c6af4b65b2f25b905461f80f13b89edd",
"7ebeae7195a9f372afdc9d94744cc349",
"5b79cf6d5a5a9bf0bb866aed449eff44")
allTaxa = taxa_names(psr_cer)
allTaxa <- allTaxa[(allTaxa %in% GoodTaxa)]
sig_cer_geno = prune_taxa(allTaxa, psr_cer)
sig_cer_geno





sum_sig_cer_geno  <- sig_cer_geno  %>%
  tax_glom(taxrank = "Family") %>%                     # agglomerate at genus level
  psmelt() %>%                                         # Melt to long format
  arrange(Family) 

GoodTaxa = c(

"8415141e03996c7309d7d1fd8aee6967",
"7253c0f3989180f5b68f83c04ecfbda2",
"7ebeae7195a9f372afdc9d94744cc349")
allTaxa = taxa_names(psr_pal)
allTaxa <- allTaxa[(allTaxa %in% GoodTaxa)]
sig_pal_geno = prune_taxa(allTaxa, psr_pal)
sig_pal_geno

sum_sig_pal_geno <- sig_pal_geno  %>%
  tax_glom(taxrank = "Family") %>%                     # agglomerate at genus level
  psmelt() %>%                                         # Melt to long format
  arrange(Family) 


In [ ]:
sum_sig_cer_geno$Susceptibility <- factor(sum_sig_cer_geno$Susceptibility, levels = c("Low",
                                                         "Mid","High"))

bar_cer_geno =ggplot(subset(sum_sig_cer_geno, Abundance > 0.0), aes(x =Genotype, y=Abundance)) + 
geom_bar(stat="identity", aes(fill = Family)) +
scale_fill_manual(values=c("#CD9BCD", "grey","#000080","#D14285", "#652926","#8569D5", 
                           "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4", 
                           "khaki1", "lightblue4", "lightpink", "ivory4", "yellow2",
                           "royalblue4", "darkorchid", "palevioletred1", "#CBD588",
                           "#5F7FC7", "orange","#DA5724","#CD9BCD", "gray80")) +
theme_classic() +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.text =element_text(size=4)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 8)) +
theme(axis.title.y = element_text(size = 8)) +
theme(axis.text.x = element_text(size =4)) +
theme(axis.text.y = element_text(size =5)) +
theme(strip.text.x = element_text(size =5)) +
guides(fill = guide_legend(keywidth = 0.2, , keyheight =.30)) +
ylab("Relative abundance")
bar_cer_geno

In [ ]:
bar_pal_geno=ggplot(subset(sum_sig_pal_geno, Abundance > 0.0), aes(x = Genotype, y=Abundance)) + 
geom_bar(stat="identity", aes(fill = Family)) +
scale_fill_manual(values=c("#CD9BCD","#000080", "orange","#DA5724","#CD9BCD", "gray80", "#AD6F3B", "#673770","#D14285", "#652926","#8569D5", "#5E738F","#D1A33D", "#8A7C64","lightsalmon","aquamarine4", "khaki1", "lightblue4", "lightpink", "ivory4", "yellow2","royalblue4", "darkorchid", "palevioletred1", "#CBD588","#5F7FC7", "orange","#DA5724","#CD9BCD", "gray80")) +
theme_classic() +
theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
theme(legend.text =element_text(size=4)) +
theme(legend.title =element_text(size=6)) +
theme(axis.title.x = element_text(size = 8)) +
theme(axis.title.y = element_text(size = 8)) +
theme(axis.text.x = element_text(size =4)) +
theme(axis.text.y = element_text(size =5)) +
theme(strip.text.x = element_text(size =5)) +
guides(fill = guide_legend(keywidth = 0.2, , keyheight =.30)) +
ylab("Relative abundance")
bar_pal_geno

In [ ]:
sup3 = ggarrange(bar_cer_geno, bar_pal_geno,
                 ncol = 2,
                  common.legend = TRUE, legend = "top",
                labels = c("A", "B"), font.label = list(size = 9) 
               )
sup3 

In [ ]:
#save figure
ggsave("microbiome/Figures/supplementalFig3.png"
       ,sup3, width = 4, height = 4, units = "in", dpi=300)

In [ ]:
#subset data by categories 
ps_T_H_AC =subset_samples(ps_cer, Outcome=="Control")
ps_T_H_AC

ps_T_D_AC =subset_samples(ps_cer, Outcome=="Diseased")
ps_T_D_AC

ps_T_U_AC =subset_samples(ps_cer, Outcome=="VisuallyUnaffected")
ps_T_U_AC


ps_T_H_AP =subset_samples(ps_pal, Outcome=="Control")
ps_T_H_AP

ps_T_D_AP =subset_samples(ps_pal, Outcome=="Diseased")
ps_T_D_AP

ps_T_U_AP =subset_samples(ps_pal, Outcome=="VisuallyUnaffected")
ps_T_U_AP


ps_H_AC =subset_samples(ps_cer, Susceptibility=="High")
ps_H_AC

ps_M_AC =subset_samples(ps_cer, Susceptibility=="Mid")
ps_M_AC


ps_L_AC =subset_samples(ps_cer, Susceptibility=="Low")
ps_L_AC


In [ ]:
#Core microbiome analysis 

print("CER_disease")
core_ps_T_D_AC=core(ps_T_D_AC, detection = 0, prevalence = .99)
core_ps_T_D_AC

print("CER_CONTROL")
core_ps_T_H_AC=core(ps_T_H_AC, detection = 0, prevalence = .99)
core_ps_T_H_AC

print("CER_VISULALLYhealthy")
core_ps_T_U_AC=core(ps_T_U_AC, detection = 0, prevalence = .99)
core_ps_T_U_AC

print("Pal_disease")
core_ps_T_D_AP=core(ps_T_D_AP, detection = 0, prevalence = .99)
core_ps_T_D_AP

print("PAL_healthy")
core_ps_T_H_AP=core(ps_T_H_AP, detection = 0, prevalence = .99)
core_ps_T_H_AP

print("PAL_VISULALLYhealthy")
core_ps_T_U_AP=core(ps_T_U_AP, detection = 0, prevalence = .99)
core_ps_T_U_AP



print("ps_H_AC")
core_ps_H_AC=core(ps_H_AC, detection = 0, prevalence = .99)
core_ps_H_AC

print("ps_M_AC")
core_ps_M_AC=core(ps_M_AC, detection = 0, prevalence = .99)
core_ps_M_AC

print("ps_L_AC")
core_ps_L_AC=core(ps_L_AC, detection = 0, prevalence = .99)
core_ps_L_AC


In [ ]:
R.version